In [19]:
# Spider

In [34]:
from requests_html import HTMLSession

sess = HTMLSession()

In [35]:
from requests_html import HTMLSession

def get_heroes(session):
    r = session.get('https://www.dotabuff.com/heroes/')
    hero_list = []
    for a in r.html.find('a'):
        if a.attrs['href'].find('heroes/') != -1:
            hero_list.append(a.attrs['href'][8:])
    return set(hero_list[10:])

def get_items(session):
    r = session.get('https://www.dotabuff.com/items/')
    item_list = []
    for a in r.html.find('a'):
        if a.attrs['href'].find('items/') != -1:
            item_list.append(a.attrs['href'][7:])
    return set(item_list[3:])

In [36]:
heroes, items = get_heroes(sess), get_items(sess)

In [23]:
def extract_hero_info(session, hero_name):
    r = session.get('https://www.dotabuff.com/heroes/' + hero_name)
    hero_labels = [l.strip() for l in r.html.find('small')[0].html[7:-8].split(',')]
    hero_prim_attr = r.html.find('tbody')[8].attrs['class'][0][8:]
    sections = next(
        filter(lambda x: x.attrs.get('class', ('',))[0] == 'col-8', 
               r.html.find('div'))
    ).find('section')
    lanes = []
    for tr in sections[0].find('tr')[1:]:
        if float(tr.find('td')[1].attrs['data-value']) > 20:
            lanes.append(tr.find('td')[0].html[4:-5])
    hero_items = []
    for tr in sections[-3].find('tr'):
        if 'data-link-to' in tr.attrs:
            hero_items.append(tr.attrs['data-link-to'][7:])
    best_against = []
    for tr in sections[-2].find('tr'):
        if 'data-link-to' in tr.attrs:
            best_against.append(tr.attrs['data-link-to'][8:])
    worst_against = []
    for tr in sections[-1].find('tr'):
        if 'data-link-to' in tr.attrs:
            worst_against.append(tr.attrs['data-link-to'][8:])
    return {
        'labels': hero_labels,
        'prim_attr': hero_prim_attr,
        'lanes': lanes,
        'items': hero_items,
        'best_vs': best_against,
        'worst_vs': worst_against
    }

In [102]:
from tqdm.notebook import tqdm

hero_info = dict()
for hero in tqdm(heroes):
    if not hero in hero_info:
        hero_info[hero] = extract_hero_info(sess, hero)

  0%|          | 0/123 [00:00<?, ?it/s]

In [24]:
from requests.exceptions import ProxyError

def get_player_list(session, max_page=158):
    players = []
    for i in tqdm(range(1, max_page + 1)):
        try:
            r = session.get('https://www.dotabuff.com/players/leaderboard?page={}'.format(i))
        except ProxyError:
            continue
        for tr in r.html.find('tbody')[0].find('tr')[1:]:
            a = tr.find('a')[0]
            pid = a.attrs['href'][9:]
            name = a.html[a.html[:-1].rfind('>') + 1:-4]
            players.append({
                'id': pid,
                'name': name
            })
    return players

In [168]:
players = get_player_list(sess)

  0%|          | 0/158 [00:00<?, ?it/s]

In [197]:
import pickle as pkl

'''
with open('players.pkl', 'wb') as f:
    pkl.dump(players, f)
with open('hero_info.pkl', 'wb') as f:
    pkl.dump(hero_info, f)   
'''

In [3]:
import pickle as pkl

with open('players.pkl', 'rb') as f:
    players = pkl.load(f)
with open('hero_info.pkl', 'rb') as f:
    hero_info = pkl.load(f)

In [38]:
import requests
import json

ret = requests.get('https://api.opendota.com/api/heroes').text

In [39]:
id2hero = {}

for hero in eval(ret):
    id2hero[hero['id']] = hero['name'][14:].replace('_', '-')

In [40]:
id2hero

{1: 'antimage',
 2: 'axe',
 3: 'bane',
 4: 'bloodseeker',
 5: 'crystal-maiden',
 6: 'drow-ranger',
 7: 'earthshaker',
 8: 'juggernaut',
 9: 'mirana',
 10: 'morphling',
 11: 'nevermore',
 12: 'phantom-lancer',
 13: 'puck',
 14: 'pudge',
 15: 'razor',
 16: 'sand-king',
 17: 'storm-spirit',
 18: 'sven',
 19: 'tiny',
 20: 'vengefulspirit',
 21: 'windrunner',
 22: 'zuus',
 23: 'kunkka',
 25: 'lina',
 26: 'lion',
 27: 'shadow-shaman',
 28: 'slardar',
 29: 'tidehunter',
 30: 'witch-doctor',
 31: 'lich',
 32: 'riki',
 33: 'enigma',
 34: 'tinker',
 35: 'sniper',
 36: 'necrolyte',
 37: 'warlock',
 38: 'beastmaster',
 39: 'queenofpain',
 40: 'venomancer',
 41: 'faceless-void',
 42: 'skeleton-king',
 43: 'death-prophet',
 44: 'phantom-assassin',
 45: 'pugna',
 46: 'templar-assassin',
 47: 'viper',
 48: 'luna',
 49: 'dragon-knight',
 50: 'dazzle',
 51: 'rattletrap',
 52: 'leshrac',
 53: 'furion',
 54: 'life-stealer',
 55: 'dark-seer',
 56: 'clinkz',
 57: 'omniknight',
 58: 'enchantress',
 59: 'husk

In [43]:
from tqdm.notebook import tqdm
from time import sleep

player_data = []
for p in tqdm(players):
    try:
        data = eval(requests.get('https://api.opendota.com/api/players/{}/heroes'.format(p['id'])).text)
    except Exception:
        data = []
        print('Failed!')
    player_data.append(data)
    sleep(0.7)

  0%|          | 0/2505 [00:00<?, ?it/s]

Failed!
Failed!
Failed!
Failed!
Failed!
Failed!
Failed!
Failed!
Failed!
Failed!
Failed!
Failed!
Failed!
Failed!
Failed!
Failed!
Failed!
Failed!
Failed!
Failed!
Failed!
Failed!
Failed!
Failed!


In [44]:
player_data

[{'hero_id': '74',
  'last_played': 1650443000,
  'games': 515,
  'win': 261,
  'with_games': 340,
  'with_win': 190,
  'against_games': 798,
  'against_win': 436},
 {'hero_id': '17',
  'last_played': 1650885039,
  'games': 512,
  'win': 299,
  'with_games': 339,
  'with_win': 197,
  'against_games': 685,
  'against_win': 401},
 {'hero_id': '11',
  'last_played': 1639738652,
  'games': 489,
  'win': 255,
  'with_games': 305,
  'with_win': 157,
  'against_games': 670,
  'against_win': 380},
 {'hero_id': '8',
  'last_played': 1630502251,
  'games': 405,
  'win': 217,
  'with_games': 698,
  'with_win': 393,
  'against_games': 1037,
  'against_win': 567},
 {'hero_id': '10',
  'last_played': 1633694560,
  'games': 383,
  'win': 222,
  'with_games': 218,
  'with_win': 118,
  'against_games': 514,
  'against_win': 295},
 {'hero_id': '109',
  'last_played': 1629111190,
  'games': 327,
  'win': 179,
  'with_games': 191,
  'with_win': 104,
  'against_games': 435,
  'against_win': 226},
 {'hero_i

In [1]:
import pickle as pkl

'''
with open('player_data.pkl', 'wb') as f:
    pkl.dump(player_data, f)
'''

"\nwith open('player_data.pkl', 'wb') as f:\n    pkl.dump(player_data, f)\n"

In [2]:
with open('player_data.pkl', 'rb') as f:
    player_data = pkl.load(f)

In [42]:
player_data[0]

[{'hero_id': '74',
  'last_played': 1650443000,
  'games': 515,
  'win': 261,
  'with_games': 340,
  'with_win': 190,
  'against_games': 798,
  'against_win': 436},
 {'hero_id': '17',
  'last_played': 1650885039,
  'games': 512,
  'win': 299,
  'with_games': 339,
  'with_win': 197,
  'against_games': 685,
  'against_win': 401},
 {'hero_id': '11',
  'last_played': 1639738652,
  'games': 489,
  'win': 255,
  'with_games': 305,
  'with_win': 157,
  'against_games': 670,
  'against_win': 380},
 {'hero_id': '8',
  'last_played': 1630502251,
  'games': 405,
  'win': 217,
  'with_games': 698,
  'with_win': 393,
  'against_games': 1037,
  'against_win': 567},
 {'hero_id': '10',
  'last_played': 1633694560,
  'games': 383,
  'win': 222,
  'with_games': 218,
  'with_win': 118,
  'against_games': 514,
  'against_win': 295},
 {'hero_id': '109',
  'last_played': 1629111190,
  'games': 327,
  'win': 179,
  'with_games': 191,
  'with_win': 104,
  'against_games': 435,
  'against_win': 226},
 {'hero_i

In [ ]:
# Create Graph Database

In [11]:
from py2neo import Graph, Node, Relationship

graph = Graph("bolt://localhost:7687", auth=("neo4j", "1234"))

In [13]:
t = graph.begin()
for h in heroes:
    n = Node('Hero', name=h)
    t.create(n)
t.commit()

<ipython-input-13-402aa1d6d8dc>:5: DeprecationWarning: The transaction.commit() method is deprecated, use graph.commit(transaction) instead
  t.commit()


In [15]:
t = graph.begin()
for i in items:
    n = Node('Item', name=i)
    t.create(n)
graph.commit(t)

In [17]:
hero_info

{'bristleback': {'labels': ['Melee', 'Carry', 'Durable', 'Initiator', 'Nuker'],
  'prim_attr': 'strength',
  'lanes': ['Off Lane'],
  'items': ['vanguard',
   'phase-boots',
   'aghanims-scepter',
   'eternal-shroud',
   'blade-mail',
   'heart-of-tarrasque',
   'crimson-guard',
   'magic-wand',
   'power-treads',
   'quelling-blade',
   'aghanims-shard',
   'soul-ring'],
  'best_vs': ['lone-druid',
   'visage',
   'lycan',
   'broodmother',
   'spectre',
   'enigma',
   'centaur-warrunner',
   'arc-warden',
   'clinkz',
   'meepo'],
  'worst_vs': ['slark',
   'viper',
   'drow-ranger',
   'silencer',
   'hoodwink',
   'legion-commander',
   'ancient-apparition',
   'anti-mage',
   'monkey-king',
   'grimstroke']},
 'kunkka': {'labels': ['Melee',
   'Carry',
   'Disabler',
   'Durable',
   'Initiator',
   'Nuker',
   'Support'],
  'prim_attr': 'strength',
  'lanes': ['Mid Lane', 'Off Lane'],
  'items': ['phase-boots',
   'bracer',
   'black-king-bar',
   'silver-edge',
   'aghanims-sce

In [19]:
t = graph.begin()
for a in ['strength', 'agility', 'intelligence']:
    n = Node('PrimaryAttribute', name=a)
    t.create(n)
graph.commit(t)

In [20]:
labels = set()
for hero, d in hero_info.items():
    labels = labels.union(set(d['labels']))

In [22]:
t = graph.begin()
for l in labels:
    n = Node('HeroLabel', name=l)
    t.create(n)
graph.commit(t)

In [23]:
lanes = set()
for hero, d in hero_info.items():
    lanes = lanes.union(set(d['lanes']))

In [25]:
t = graph.begin()
for l in lanes:
    n = Node('Lane', name=l)
    t.create(n)
graph.commit(t)

In [44]:
t = graph.begin()
for name, info in hero_info.items():
    h = graph.nodes.match('Hero', name=name).first()
    pa = graph.nodes.match('PrimaryAttribute', name=info['prim_attr']).first()
    r = Relationship(h, 'PrimaryAttributeIs', pa)
    t.create(r)
graph.commit(t)

In [45]:
t = graph.begin()
for name, info in hero_info.items():
    h = graph.nodes.match('Hero', name=name).first()
    for l_name in info['labels']:
        l = graph.nodes.match('HeroLabel', name=l_name).first()
        r = Relationship(h, 'HasLabel', l)
        t.create(r)
graph.commit(t)

In [47]:
t = graph.begin()
for name, info in hero_info.items():
    h = graph.nodes.match('Hero', name=name).first()
    for i_name in info['items']:
        i = graph.nodes.match('Item', name=i_name).first()
        r = Relationship(h, 'UsesItem', i)
        t.create(r)
graph.commit(t)

In [49]:
t = graph.begin()
for name, info in hero_info.items():
    h = graph.nodes.match('Hero', name=name).first()
    for l_name in info['lanes']:
        l = graph.nodes.match('Lane', name=l_name).first()
        r = Relationship(h, 'AtLane', l)
        t.create(r)
graph.commit(t)

In [50]:
t = graph.begin()
for name, info in hero_info.items():
    h = graph.nodes.match('Hero', name=name).first()
    for name2 in info['best_vs']:
        h2 = graph.nodes.match('Hero', name=name2).first()
        r = Relationship(h, 'BestAgainst', h2)
        t.create(r)
    for name2 in info['worst_vs']:
        h2 = graph.nodes.match('Hero', name=name2).first()
        r = Relationship(h, 'WorstAgainst', h2)
        t.create(r)
graph.commit(t)

In [ ]:
wrong_names = [('outworld-destroyer', 'obsidian-destroyer'), 
               ('necrophos', 'necrolyte'), 
               ('magnus', 'magnataur'), 
               ('shadow-fiend', 'nevermore'), 
               ('queen-of-pain', 'queenofpain'), 
               ('zeus', 'zuus'), 
               ('wind-ranger', 'windrunner'), 
               ('io', 'wisp'), 
               ('underlord', 'abyssal-underlord'),
               ('centaur-warrunner', 'centaur'),
               ('natures-prophet', 'furion'),
               ('timbersaw', 'shredder'),
               ('clockwerk', 'rattletrap'),
               ('vengeful-spirit', 'vengefulspirit'),
               ('doom-', 'doom-bringer'),
               ('treant', 'treant-protector'),
               ('wraith-king', 'skeleton-king'),
               ('lifestealer', 'life-stealer'),
               ('anti-mage', 'antimage')]

for dotabuff_name, opendota_name in wrong_names:
    node = graph.nodes.match('Hero', name=dotabuff_name).first()
    node['name'] = opendota_name
    graph.push(node)

In [51]:
# User Data For Reccomendation

In [4]:
res = list(filter(lambda x:len(x[1]), zip(players, player_data)))

In [48]:
import numpy as np

hero2id = {}
for i, name in enumerate(id2hero.values()):
    hero2id[name] = i
players = [t[0] for t in res]
games = np.zeros((len(res), len(heroes)), dtype=np.float32)
win = np.zeros((len(res), len(heroes)), dtype=np.float32)
for uid, (_, l) in enumerate(res):
    for d in l:
        hid = hero2id[id2hero[int(d['hero_id'])]]
        games[uid][hid] = d['games']
        win[uid][hid] = d['win']
data4rec = {
    'hero2id': hero2id,
    'players': players,
    'games': games,
    'win': win
}

In [49]:
data4rec

{'hero2id': {'antimage': 0,
  'axe': 1,
  'bane': 2,
  'bloodseeker': 3,
  'crystal-maiden': 4,
  'drow-ranger': 5,
  'earthshaker': 6,
  'juggernaut': 7,
  'mirana': 8,
  'morphling': 9,
  'nevermore': 10,
  'phantom-lancer': 11,
  'puck': 12,
  'pudge': 13,
  'razor': 14,
  'sand-king': 15,
  'storm-spirit': 16,
  'sven': 17,
  'tiny': 18,
  'vengefulspirit': 19,
  'windrunner': 20,
  'zuus': 21,
  'kunkka': 22,
  'lina': 23,
  'lion': 24,
  'shadow-shaman': 25,
  'slardar': 26,
  'tidehunter': 27,
  'witch-doctor': 28,
  'lich': 29,
  'riki': 30,
  'enigma': 31,
  'tinker': 32,
  'sniper': 33,
  'necrolyte': 34,
  'warlock': 35,
  'beastmaster': 36,
  'queenofpain': 37,
  'venomancer': 38,
  'faceless-void': 39,
  'skeleton-king': 40,
  'death-prophet': 41,
  'phantom-assassin': 42,
  'pugna': 43,
  'templar-assassin': 44,
  'viper': 45,
  'luna': 46,
  'dragon-knight': 47,
  'dazzle': 48,
  'rattletrap': 49,
  'leshrac': 50,
  'furion': 51,
  'life-stealer': 52,
  'dark-seer': 53,


In [50]:
import pickle as pkl

with open('data4rec.pkl', 'wb') as f:
    pkl.dump(data4rec, f)

In [6]:
res[0]

({'id': '137193239', 'name': 'Elephant.eurus'},
 [{'hero_id': '74',
   'last_played': 1650443000,
   'games': 515,
   'win': 261,
   'with_games': 340,
   'with_win': 190,
   'against_games': 798,
   'against_win': 436},
  {'hero_id': '17',
   'last_played': 1650885039,
   'games': 512,
   'win': 299,
   'with_games': 339,
   'with_win': 197,
   'against_games': 685,
   'against_win': 401},
  {'hero_id': '11',
   'last_played': 1639738652,
   'games': 489,
   'win': 255,
   'with_games': 305,
   'with_win': 157,
   'against_games': 670,
   'against_win': 380},
  {'hero_id': '8',
   'last_played': 1630502251,
   'games': 405,
   'win': 217,
   'with_games': 698,
   'with_win': 393,
   'against_games': 1037,
   'against_win': 567},
  {'hero_id': '10',
   'last_played': 1633694560,
   'games': 383,
   'win': 222,
   'with_games': 218,
   'with_win': 118,
   'against_games': 514,
   'against_win': 295},
  {'hero_id': '109',
   'last_played': 1629111190,
   'games': 327,
   'win': 179,
   '

In [12]:
import pickle as pkl

with open('player_composed.pkl', 'wb') as f:
    pkl.dump(res, f)

In [17]:
import numpy as np
np.linspace(1,3,3)

array([1., 2., 3.])

In [22]:
np.where(np.array([1,2,3]) == 1)[0].dtype

dtype('int64')

In [23]:
from functools import reduce
reduce(lambda x, y: x + y, [1,2,3])

6